In [96]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from tabulate import tabulate

In [97]:
df = pd.read_excel('Cisco Forecast League Data Pack - MIT.xlsx', header=2)

In [98]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,FY20 Q4,FY21 Q1,FY21 Q2,FY21 Q3,FY21 Q4,FY22 Q1,FY22 Q2,FY22 Q3,FY22 Q4,FY23 Q1,FY23 Q2,FY23 Q3,FY23 Q4,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,ROUTER 1,Sustaining,NaN,NaN,NaN,906.0,941.0,663.0,769.0,3565.0,1020,1670,38,2141,NaN,1716.00,1716.0000,1948
1,2,TRANSCIEVER,Sustaining,87173.0,102849.0,126375.0,128629.0,179125.0,138750.0,208760.0,116126.0,150803,82163,82408,67132,NaN,85000.00,141285.0000,97995
2,3,SWITCH 1,Sustaining,NaN,NaN,NaN,3651.0,10409.0,10575.0,17934.0,15104.0,15525,6938,9595,9231,NaN,12000.00,11819.7135,11184
3,4,ACCESS POINT 1,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,9576.0,46029.0,109452,54955,60478,62708,NaN,62770.99,79113.0000,65310
4,5,ACCESS POINT 2,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26769,33670,74853,80944,NaN,76191.97,113704.0000,93200


In [99]:
df.columns = [col.strip() for col in df.columns]

In [100]:
df_melted = df.melt(id_vars=["Unnamed: 1", "Unnamed: 2"], 
                    var_name="Quarter", 
                    value_name="Units")

In [101]:
df_melted = df_melted[df_melted['Quarter'].str.startswith('FY')]

In [102]:
df_melted['Quarter'] = df_melted['Quarter'].str.replace('FY', '20').str.replace(' ', '')

In [103]:
df_melted['Quarter'] = pd.PeriodIndex(df_melted['Quarter'], freq='Q')

In [104]:
df_pivot = df_melted.pivot_table(index="Quarter", columns="Unnamed: 1", values="Units")

In [105]:
print(df_pivot.columns)

Index(['ACCESS POINT 1', 'ACCESS POINT 2', 'ACCESS POINT 3', 'ACCESS POINT 4',
       'POWER SUPPLY 1', 'POWER SUPPLY 2', 'ROUTER 1', 'SUPERVISOR ENGINE',
       'SWITCH 1', 'SWITCH 10', 'SWITCH 2', 'SWITCH 3', 'SWITCH 4', 'SWITCH 5',
       'SWITCH 6', 'SWITCH 7', 'SWITCH 8', 'SWITCH 9', 'TRANSCIEVER',
       'WIRELESS CONTROLLER'],
      dtype='object', name='Unnamed: 1')


In [106]:
holt_forecasts = {}
holt_mae_values = {}
holt_me_values = {}
holt_bias_values = {}
holt_model_used = {}

In [107]:
arima_forecasts = {}
arima_mae_values = {}
arima_me_values = {}
arima_bias_values = {}
arima_model_used = {}

In [108]:
for column_name in df_pivot.columns:
    product_series = df_pivot[column_name].dropna()
    
    if len(product_series) > 2:
        try:
            model_holt = ExponentialSmoothing(product_series, trend='add', seasonal=None, initialization_method="estimated").fit()
            forecast_holt = model_holt.forecast(steps=1)
            holt_forecasts[column_name] = forecast_holt.iloc[0]
            
            fitted_values_holt = model_holt.fittedvalues
            errors_holt = product_series - fitted_values_holt
            mae_holt = errors_holt.abs().mean()
            me_holt = errors_holt.mean()
            bias_holt = (me_holt / product_series.mean()) * 100  # Bias percentage
            
            holt_mae_values[column_name] = mae_holt
            holt_me_values[column_name] = me_holt
            holt_bias_values[column_name] = bias_holt
            holt_model_used[column_name] = "Holt's Exponential Smoothing"
        except:
            print(f"Holt's Exponential Smoothing model failed for {column_name}")

    if len(product_series) > 2:
        try:
            model_arima = ARIMA(product_series, order=(1, 1, 1)).fit()
            forecast_arima = model_arima.forecast(steps=1)
            arima_forecasts[column_name] = forecast_arima.iloc[0]
            
            fitted_values_arima = model_arima.fittedvalues
            errors_arima = product_series - fitted_values_arima
            mae_arima = errors_arima.abs().mean()
            me_arima = errors_arima.mean()
            bias_arima = (me_arima / product_series.mean()) * 100  # Bias percentage
            
            arima_mae_values[column_name] = mae_arima
            arima_me_values[column_name] = me_arima
            arima_bias_values[column_name] = bias_arima
            arima_model_used[column_name] = "ARIMA"
        except:
            print(f"ARIMA model failed for {column_name}")

C:\Users\Vinayak\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Vinayak\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Vinayak\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Vinayak\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\Vinayak\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization fail

In [109]:
df_holt_forecasts = pd.DataFrame.from_dict(holt_forecasts, orient='index', columns=['Forecast'])
df_holt_mae = pd.Series(holt_mae_values, name='Mean Absolute Error (MAE)')
df_holt_me = pd.Series(holt_me_values, name='Mean Error (ME)')
df_holt_bias = pd.Series(holt_bias_values, name='Bias (%)')
df_holt_model = pd.Series(holt_model_used, name='Model Used')
df_holt_combined = pd.concat([df_holt_forecasts, df_holt_mae, df_holt_me, df_holt_bias, df_holt_model], axis=1)

In [110]:
df_arima_forecasts = pd.DataFrame.from_dict(arima_forecasts, orient='index', columns=['Forecast'])
df_arima_mae = pd.Series(arima_mae_values, name='Mean Absolute Error (MAE)')
df_arima_me = pd.Series(arima_me_values, name='Mean Error (ME)')
df_arima_bias = pd.Series(arima_bias_values, name='Bias (%)')
df_arima_model = pd.Series(arima_model_used, name='Model Used')
df_arima_combined = pd.concat([df_arima_forecasts, df_arima_mae, df_arima_me, df_arima_bias, df_arima_model], axis=1)

In [111]:
print("Results for Holt's Exponential Smoothing:")
print(tabulate(df_holt_combined, headers='keys', tablefmt='plain'))

Results for Holt's Exponential Smoothing:
                       Forecast    Mean Absolute Error (MAE)    Mean Error (ME)    Bias (%)  Model Used
ACCESS POINT 1         63546.4                     34011.4         -21194        -37.0526    Holt's Exponential Smoothing
ACCESS POINT 2         87842.9                     10385.3           6836.57      12.6465    Holt's Exponential Smoothing
ACCESS POINT 3         63263.9                     40710.2         -13498.3      -10.0357    Holt's Exponential Smoothing
ACCESS POINT 4         48576.6                       808.187         -107.034     -0.404169  Holt's Exponential Smoothing
POWER SUPPLY 1         75817.4                     21970.7          -5999.52      -3.7287    Holt's Exponential Smoothing
POWER SUPPLY 2         21141.5                      7568.76         -2357.23      -7.48018   Holt's Exponential Smoothing
ROUTER 1                1651.32                      682.017           -6.39274   -0.491204  Holt's Exponential Smoothing


In [112]:
print("\nResults for ARIMA:")
print(tabulate(df_arima_combined, headers='keys', tablefmt='plain'))


Results for ARIMA:
                       Forecast    Mean Absolute Error (MAE)    Mean Error (ME)    Bias (%)  Model Used
ACCESS POINT 1         57061.5                     24110.1            22410.8      39.18     ARIMA
ACCESS POINT 2         97294.3                     19056.5             9219.31     17.0542   ARIMA
ACCESS POINT 3         57038.7                     42131.4             3671.8       2.7299   ARIMA
ACCESS POINT 4         49577.7                      3847.8             3539.89     13.3669   ARIMA
POWER SUPPLY 1         92504.3                     30014               3105.56      1.93011  ARIMA
POWER SUPPLY 2         24051.9                      7769.23            1998.43      6.34159  ARIMA
ROUTER 1                1004.83                      764.147            501.511    38.535    ARIMA
SUPERVISOR ENGINE       2097.49                      597.471            348.464    22.1036   ARIMA
SWITCH 1               11535.2                      3446.64            1350.57     1